# Практическая работа 1. Источники данных. Извлечение данных



ФИО: Шепелева Елизавета

Группа: АДЭУ-201

Используем данные из открытого источника для `CSV`, `Parquet` и `API`, а также вручную подготовим данные для баз данных и HTML  из открытых данных (доступно по адресу https://data.cityofnewyork.us).

In [ ]:
# import modules
import certifi
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import urllib3
from urllib3 import request
from unicodedata import normalize

1.1 Файлы данных `CSV` и `Excel`

Файлы данных могут быть получены откуда угодно: от файлов, хранящихся локально на вашем устройстве, до файловых систем облачного хранилища.

Даже при работе в основном с базами данных или внешними `API` использование физических файлов — отличный способ легко использовать данные с метками времени, что может пригодиться при любых временных проблемах с подключением.

In [ ]:
# Загрузите данные с портала открытых данных Правительства Москвы —
# данные о Wi-Fi в кинотеатрах, введя в браузер следующий URL-адрес: https://data.mos.ru/opendata/60789?isDynamic=false.
# Будет загружен CSV-файл *.csv. Сохраните этот файл.
# прочитать данные CSV, используя команду Pandas pd.read_csv() в качестве df DataFrame, затем выберите первые пять строк DataFrame с помощью df.head().

**Задание 1.1.** Зайти на сайт URL-адрес: https://data.mos.ru/opendata/60789?isDynamic=false

**Задание 1.2.** Загрузить данные о `Wi-Fi в кинотеатрах` в Colab.

In [ ]:
from google.colab import files

uploaded = files.upload()

**Задание 1.3.** Считайте данные из файла CSV. Использовать метод `pandas read_csv` для простоты и скорости.

In [ ]:
df_csv = pd.read_csv("data-60789-2024-01-24.csv", delimiter=';')
df_csv.head()

**Задание 1.4.** Так как первая строка включает определение столбцов, удалить строку с `index=0`.

In [ ]:
data = df_csv.drop(df_csv.index[0])

In [ ]:
data

# **Задание 1.5.** Выполнить парсинг веб-страницы с помощью `JSON`

In [ ]:
# Импорт стандартных пакетов
import pandas as pd

# Для парсинга веб-страниц
import requests as reqs
import urllib.request
from bs4 import BeautifulSoup
import json
import os
import pprint

# Для выполнения операций с регулярными выражениями
import re

api_url = 'https://jsonplaceholder.typicode.com/todos/1'

In [ ]:
resp = reqs.get(api_url)

## <font color='blue'>Парсинг веб-страниц с помощью JSON</font>

### <font color="green"> Что такое JSON?</font>

* JSON (нотация объектов JavaScript) — популярный формат данных, используемый для представления структурированных данных.
* Это текстовый формат, который не зависит от языка и может использоваться в Python, Perl и других языках.
* Формат JSON используется для передачи данных между серверами и веб-приложениями.
* Он построен на двух структурах:

      - Коллекция пар имя/значение. Это реализуется как объект, запись, словарь, хеш-таблица, список ключей или ассоциативный массив.
      - Упорядоченный список значений. Это реализуется как массив, вектор, список или последовательность.
     
     
Основные функции `JSON`:

* `dump()`: запись закодированной строки в файл.
* `load()`: декодирование во время чтения файла JSON.
* `dumps()`: кодирование в объекты JSON.
* `loads()`: декодирует строку JSON.

### <font color="green"> Сериализация и десериализация</font>

> … процесс перевода структур данных или состояния объекта в формат, который можно сохранить… или передать… и позже восстановить (возможно, в другой компьютерной среде). (Википедия)

* **Сериализация** — это процесс преобразования объектов или структур данных в потоки байтов или строки.
* Эти потоки байтов затем можно легко сохранить или передать.
* Это позволяет разработчикам сохранять, например, данные конфигурации или прогресс пользователя, а затем сохранять их (на диске или в базе данных) или отправлять в другое место.
* Обратный процесс сериализации известен как **десериализация**.

### Зачем нужна сериализация?

Нам нужна сериализация по следующим причинам:

- **Связь**: Сериализация включает в себя процедуру сериализации и передачи объекта. Это позволяет нескольким компьютерным системам одновременно проектировать, совместно использовать и выполнять объекты.
- **Кэширование**: время, затрачиваемое на создание объекта, больше времени, необходимого для его десериализации. Сериализация минимизирует затраты времени за счет кэширования гигантских объектов.
- **Глубокое копирование**: процесс клонирования упрощается благодаря сериализации. Точная копия объекта получается путем сериализации объекта в массив байтов и последующей его десериализации.
- **Переносимость**. Основное преимущество сериализации заключается в том, что она работает в разных архитектурах и операционных системах.
- **Постоянство**: состояние любого объекта можно сохранить напрямую, применив к нему сериализацию, и сохранить в базе данных, чтобы его можно было получить позже.

**Сериализация**

Мы используем функцию dump(), которая принимает два аргумента:
* Объект данных, подлежащий сериализации.
* Объект файла, в который он будет записан (байтовый формат).

In [ ]:
file_name = "Sample.json"
with open(file_name, "w") as fid:
     json.dump(resp.json(), fid)

In [ ]:
!cat Sample.json

{"userId": 1, "id": 1, "title": "delectus aut autem", "completed": false}

**Десериализация JSON**

* Десериализация противоположна сериализации, т. е. преобразованию объекта JSON в соответствующие объекты Python.
* Мы используем функцию load(), которая обычно используется для загрузки из строки, в противном случае корневой объект находится в списке или диктовке.

In [ ]:
with open(file_name, "r") as fid:
     json_obj = json.load(fid)

print(json_obj)

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}


In [ ]:
for key in json_obj:
    print("{:>12}: {}".format(key, json_obj[key]))

      userId: 1
          id: 1
       title: delectus aut autem
   completed: False


Обрабатываем данные с помощью `JSON`:

In [ ]:
response = reqs.get(api_url)

In [ ]:
if response.status_code == 200:
   json_page = json.loads(response.text)

**Задание 1.6.** Выполнить парсинг веб-страницы с помощью `JSON`. Получите URL-адрес:

In [ ]:
print(f"URL: \n\t {response.url}")

**Задание 1.7.** Выполнить парсинг веб-страницы с помощью `JSON`. Определите тип контента:

In [ ]:
print(f"Content type: \n\t {response.headers['content-type']}")

**Задание 1.8.** Выполнить парсинг веб-страницы с помощью `JSON`. Обработайте данные с помощью JSON:

In [ ]:
if response.status_code == 200:
   json_page = json.loads(response.text)

**Задание 1.9.**  Обработайте данные с помощью `JSON`. Посмотреть на ключи:

In [ ]:
for x in json_page:
    print(x)

Распечатать ключи и значения:

In [ ]:
for x in json_page:
    print(f"{x} --> {json_page[x]} \n")

In [ ]:
pprint.pprint(json_page)

# **Задание 1.10.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`

 <font color='blue'>Парсинг веб-страниц с помощью Beautiful Soup</font>

- Веб-скрапинг позволяет загрузить HTML-код веб-сайта и извлечь необходимые вам данные.
- Beautiful Soup — это библиотека Python для очистки данных с веб-сайтов.
- Beautiful Soup создает дерево разбора из проанализированных документов HTML и XML.

In [ ]:
from bs4 import BeautifulSoup as bso

In [ ]:
# Отправление запроса на получение данных и присваивание ответ переменной.
source = reqs.get(api_url)

In [ ]:
print(f"Content type: \n\t {source.headers['content-type']}")

In [ ]:
print(source)

**Задание 1.10.1.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Создать beautiful soup объект.

In [ ]:
mysoup = bso(source.text, 'html.parser')

**Задание 1.10.2.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Распечатайте HTML-содержимое страницы, используя метод prettify

In [ ]:
print(mysoup.prettify())

**Задание 1.10.3.** Выполнить парсинг веб-страницы с помощью `Beautiful Soup`. Получить заголовок страницы

In [ ]:
print(mysoup.title)

# Получение данных из таблиц РСУБД

In [ ]:
from google.colab import files

uploaded = files.upload()

**Задание 1.11.** Считать результаты запроса sqlite в DataFrame pandas

In [ ]:
# Считываем результаты запроса sqlite в DataFrame pandas
with sqlite3.connect("movies.sqlite") as conn:
    df = pd.read_sql("SELECT * from movies", conn)
df.head()

# Получение данных `Parquet`

Посетите URL https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

In [ ]:
from google.colab import files

uploaded = files.upload()

**Задание 1.12.** Считайте данные из файла Parquet. Использовать метод pandas read_parquet.

In [ ]:
df_parquet = pd.read_parquet("yellow_tripdata_2023-03.parquet")
df_parquet.head()